<a href="https://colab.research.google.com/github/clashleyuncc/Intro-To-ML/blob/main/HW7P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# P1a
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
import time
start_time = time.time()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(8 * 8 * 8, 32)
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
    out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
    out = out.view(-1, 8 * 8 * 8)
    out = torch.relu(self.fc1(out))
    out = self.fc2(out)
    return out

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs+1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      imgs = imgs.to(device=device)
      labels = labels.to(device=device)
      outputs = model(imgs)
      loss = loss_fn(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()
    if epoch == 1 or epoch % 10 == 0:
      print('{} Seconds, Epoch {}, Training Loss {}'.format(
          time.time()-start_time, epoch,
          loss_train / len(train_loader)))

from torchvision import transforms

tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                                  transform=transforms.ToTensor())
tensor_cifar10_val = datasets.CIFAR10(data_path, train=False, download=False,
                                  transform=transforms.ToTensor())

transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.4915, 0.4823, 0.4468),
                            (0.2470, 0.2435, 0.2616))
    ]))

tensor_cifar10 = transformed_cifar10

model = Net().to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64,
                                           shuffle=True)

training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

def validate(model, train_loader, val_loader):
  for name, loader in [("train", train_loader), ("val", val_loader)]:
    correct = 0
    total = 0

  with torch.no_grad():
    for imgs, labels in val_loader:
      imgs, labels = imgs.to(device), labels.to(device)
      outputs = model(imgs)
      _, predicted = torch.max(outputs, dim=1)
      total += labels.shape[0]
      correct += int((predicted == labels).sum())
  print("Accuracy {}: {:.2f}".format(name, correct/total))

train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(tensor_cifar10_val, batch_size=64,
                                         shuffle=False)

validate(model, train_loader, val_loader)

print(f"Training Time: {time.time()-start_time} seconds")
print(f"Approximatly: {round((time.time()-start_time)/60,3)} minutes")


Using device: cuda
Files already downloaded and verified
Files already downloaded and verified
17.058840036392212 Seconds, Epoch 1, Training Loss 2.1038315774839553
146.30653476715088 Seconds, Epoch 10, Training Loss 1.1588254480258278
287.3507544994354 Seconds, Epoch 20, Training Loss 0.9889371754873134
430.0442945957184 Seconds, Epoch 30, Training Loss 0.8991482885521086
574.4116680622101 Seconds, Epoch 40, Training Loss 0.8426430898782847
719.4079780578613 Seconds, Epoch 50, Training Loss 0.8006435016079632
866.299289226532 Seconds, Epoch 60, Training Loss 0.7685819520517383
1011.2978353500366 Seconds, Epoch 70, Training Loss 0.7445325304937485
1154.0327427387238 Seconds, Epoch 80, Training Loss 0.7226013745112188
1300.1680233478546 Seconds, Epoch 90, Training Loss 0.7038538263505681
1443.309053182602 Seconds, Epoch 100, Training Loss 0.6870795959401923
1584.1518199443817 Seconds, Epoch 110, Training Loss 0.6703655754818636
1727.0590357780457 Seconds, Epoch 120, Training Loss 0.6598

In [ ]:
# P1b
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
import time
start_time = time.time()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(8 * 8 * 8, 32)
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
    out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
    out = out.view(-1, 8 * 8 * 8)
    out = torch.relu(self.fc1(out))
    out = self.fc2(out)
    return out

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs+1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      imgs = imgs.to(device=device)
      labels = labels.to(device=device)
      outputs = model(imgs)
      loss = loss_fn(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()
    if epoch == 1 or epoch % 10 == 0:
      print('{} Seconds, Epoch {}, Training Loss {}'.format(
          time.time()-start_time, epoch,
          loss_train / len(train_loader)))

from torchvision import transforms

tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                                  transform=transforms.ToTensor())
tensor_cifar10_val = datasets.CIFAR10(data_path, train=False, download=False,
                                  transform=transforms.ToTensor())

transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.4915, 0.4823, 0.4468),
                            (0.2470, 0.2435, 0.2616))
    ]))

tensor_cifar10 = transformed_cifar10

model = Net().to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64,
                                           shuffle=True)

training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

def validate(model, train_loader, val_loader):
  for name, loader in [("train", train_loader), ("val", val_loader)]:
    correct = 0
    total = 0

    with torch.no_grad():
      for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
    print("Accuracy {}: {:.2f}".format(name, correct/total))

train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(tensor_cifar10_val, batch_size=64,
                                         shuffle=False)

validate(model, train_loader, val_loader)

print(f"Training Time: {time.time()-start_time} seconds")
print(f"Approximatly: {round((time.time()-start_time)/60,3)} minutes")
